# Worksheet 08

Name: Sai Tejaswini Junnuri
UID: U92492167

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[5.631656907965602, 6.455471531256049, 5.897545258596165, 5.851841932657325, 6.702688469530699, 5.033000126185015, 5.253077610293628, 5.687990755551044, 5.315741231427446, 4.629531287319039]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
c2 = np.random.normal(8 , 1 , 10).tolist()
print(c2)

[8.433503074928081, 7.231005996233987, 6.549512966330348, 8.737042281231167, 9.251752946458497, 6.882474269560587, 9.307517514871488, 6.422682409750532, 9.389202143213673, 8.318129271822214]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[8.318129271822214, 4.629531287319039, 9.389202143213673, 5.315741231427446, 6.422682409750532, 9.307517514871488, 6.882474269560587, 5.687990755551044, 5.253077610293628, 5.033000126185015]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

1. Number of components(k) : 2
2. Weights (assuming a fair coin) : P(S_1) = P(S_2) = 0.5
3. Mean and St.dev of both distributions

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [6]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2))]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x: (x-mean[1])**2, c2))]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.629531287319039, 5.315741231427446, 6.422682409750532, 6.882474269560587, 5.687990755551044, 5.253077610293628, 5.033000126185015]
[8.318129271822214, 9.389202143213673, 9.307517514871488]
P(C_1) = 0.7,  P(C_2) = 0.3
mean_1 = 5.60349967001247,  mean_2 = 9.004949643302458
var_1 = 0.5419514014025416,  var_2 = 0.7109195232740926


1. For the 1st distribution, mean_true = 5, var_true = 1, whereas here we got mean = 5.6 and variance = 0.54
2. For the 2nd distribution, mean_true = 8, var_true = 1, whereas here we got mean = 9.0 and variance = 0.71
3. The means are close whereas the variance for both the distributions have decreased
4. The proportion of points in each cluster is skewed and not 0.5

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [7]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    c0_x = (pdf_i[0] * prob_c[0]) / prob_x
    c1_x = (pdf_i[1] * prob_c[1]) / prob_x
    prob_c0_x.append(c0_x)
    prob_c1_x.append(c1_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  8.318129271822214
probability of observing that point if it came from cluster 0 =  2.6225742845876524e-06
probability of observing that point if it came from cluster 1 =  0.3518963944267924
point =  4.629531287319039
probability of observing that point if it came from cluster 0 =  0.14642600757709423
probability of observing that point if it came from cluster 1 =  3.340207282058079e-09
point =  9.389202143213673
probability of observing that point if it came from cluster 0 =  1.8677750149414042e-11
probability of observing that point if it came from cluster 1 =  0.4848998070242501
point =  5.315741231427446
probability of observing that point if it came from cluster 0 =  0.6393375670792141
probability of observing that point if it came from cluster 1 =  7.970041562851237e-07
point =  6.422682409750532
probability of observing that point if it came from cluster 0 =  0.23486593729177488
probability of observing that point if it came from cluster 1 =  0.0007658572587783837
point 

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [9]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/len(prob_c1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]
var = [sum([(x[0] * (x[1] - mean[0])**2) for x in zip(prob_c0_x, data)]) / sum(prob_c0_x) , sum([(x[0] * (x[1] - mean[1])**2) for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.694078721095494,  P(C_2) = 0.30592127890450604
mean_1 = 5.592685911149935,  mean_2 = 8.963647076526854
var_1 = 0.5327129059852579,  var_2 = 0.3189014497440749


Old values: 

P(C_1) = 0.7,  P(C_2) = 0.3
mean_1 = 5.60349967001247,  mean_2 = 9.004949643302458
var_1 = 0.5419514014025416,  var_2 = 0.7109195232740926

New values: 

P(C_1) = 0.694078721095494,  P(C_2) = 0.30592127890450604
mean_1 = 5.592685911149935,  mean_2 = 8.963647076526854
var_1 = 0.5327129059852579,  var_2 = 0.3189014497440749

We see that the variances of the second distribution has reduced

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [10]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    c0_x = (pdf_i[0] * prob_c[0]) / prob_x
    c1_x = (pdf_i[1] * prob_c[1]) / prob_x
    prob_c0_x.append(c0_x)
    prob_c1_x.append(c1_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()

point =  8.318129271822214
probability of observing that point if it came from cluster 0 =  1.550858514868254e-06
probability of observing that point if it came from cluster 1 =  0.16125920121014156
point =  4.629531287319039
probability of observing that point if it came from cluster 0 =  0.14607562933922355
probability of observing that point if it came from cluster 1 =  9.73209066829592e-41
point =  9.389202143213673
probability of observing that point if it came from cluster 0 =  7.004213138190253e-12
probability of observing that point if it came from cluster 1 =  0.5135379563854235
point =  5.315741231427446
probability of observing that point if it came from cluster 0 =  0.6542266372357117
probability of observing that point if it came from cluster 1 =  4.82505913254134e-29
point =  6.422682409750532
probability of observing that point if it came from cluster 0 =  0.22247622757477512
probability of observing that point if it came from cluster 1 =  2.0475939888176903e-14
point = 

The probabilities are coverging towards on dist more than the other here

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [11]:
hard_assignments = [0 if prob_c0_x[i] > prob_c1_x[i] else 1 for i in range(len(data))]
print(data, hard_assignments)

[8.318129271822214, 4.629531287319039, 9.389202143213673, 5.315741231427446, 6.422682409750532, 9.307517514871488, 6.882474269560587, 5.687990755551044, 5.253077610293628, 5.033000126185015] [1, 0, 1, 0, 0, 1, 0, 0, 0, 0]
